In [2]:
!pip install pymongo


In [6]:
from pymongo import MongoClient

# Connexion à MongoDB
client = MongoClient("mongodb://localhost:27017/")

# Création ou accès à la base de données
db = client["magasin"]  

# Création ou accès à la collection
collection = db["avis_clients"]  


In [4]:
import random
from datetime import datetime, timedelta

clients = [f"C{str(i).zfill(3)}" for i in range(1, 11)]  
produits = [f"P{str(i).zfill(3)}" for i in range(1, 6)]  

avis_textes = {
    5: "Excellent",
    4: "Très bon",
    3: "Bon",
    2: "Moyen",
    1: "Mauvais"
}

avis_clients = []
start_date = datetime(2024, 1, 1)

for i in range(50):
    note = round(random.uniform(1, 5))  # Note entière entre 1 et 5
    avis = {
        "client_id": random.choice(clients),
        "produit_id": random.choice(produits),
        "note": note,
        "commentaire": avis_textes[note],
        "date": (start_date + timedelta(days=random.randint(0, 90))).strftime("%Y-%m-%d")  # Date aléatoire sur 3 mois
    }
    avis_clients.append(avis)

# Affichage des 5 premiers avis pour aperçu
for avis in avis_clients[:5]:
    print(avis)


{'client_id': 'C006', 'produit_id': 'P005', 'note': 5, 'commentaire': 'Excellent', 'date': '2024-01-12'}
{'client_id': 'C009', 'produit_id': 'P003', 'note': 5, 'commentaire': 'Excellent', 'date': '2024-01-27'}
{'client_id': 'C008', 'produit_id': 'P001', 'note': 3, 'commentaire': 'Bon', 'date': '2024-03-01'}
{'client_id': 'C004', 'produit_id': 'P004', 'note': 5, 'commentaire': 'Excellent', 'date': '2024-02-04'}
{'client_id': 'C008', 'produit_id': 'P004', 'note': 2, 'commentaire': 'Moyen', 'date': '2024-02-29'}


In [5]:
# Insérer les avis dans la collection MongoDB
collection.insert_many(avis_clients)

print("Données insérées avec succès !")


Données insérées avec succès !


In [7]:
# Récupérer les avis depuis MongoDB
avis_recuperes = list(collection.find({}, {"_id": 0}))  # Exclure l'ID MongoDB

# Conversion en liste de tuples
avis_tuples = [(avis["client_id"], avis["produit_id"], avis["note"], avis["commentaire"], avis["date"]) for avis in avis_recuperes]

# Afficher un aperçu
for avis in avis_tuples[:5]:
    print(avis)
    

('C006', 'P005', 5, 'Excellent', '2024-01-12')
('C009', 'P003', 5, 'Excellent', '2024-01-27')
('C008', 'P001', 3, 'Bon', '2024-03-01')
('C004', 'P004', 5, 'Excellent', '2024-02-04')
('C008', 'P004', 2, 'Moyen', '2024-02-29')


In [13]:
import pandas as pd

# Convertir la liste de tuples en DataFrame
df = pd.DataFrame(avis_tuples, columns=["client_id", "produit_id", "note", "commentaire", "date"])

# Afficher les 5 premières lignes pour vérifier
df.head()



,client_id,produit_id,note,commentaire,date
0,C006,P005,5,Excellent,2024-01-12
1,C009,P003,5,Excellent,2024-01-27
2,C008,P001,3,Bon,2024-03-01
3,C004,P004,5,Excellent,2024-02-04
4,C008,P004,2,Moyen,2024-02-29


In [14]:
#Produit avec la meilleure moyenne 

meilleur_produit = df.groupby("produit_id")["note"].mean().idxmax()
meilleure_note = df.groupby("produit_id")["note"].mean().max()

print(f"Le produit {meilleur_produit} a la meilleure note moyenne de {meilleure_note:.2f}")



Le produit P003 a la meilleure note moyenne de 4.00


In [15]:
#Clinet qui a laissé le plus d'avis
meilleur_client = df["client_id"].value_counts().idxmax()
nb_avis_client = df["client_id"].value_counts().max()

print(f"Le client {meilleur_client} a laissé le plus d’avis : {nb_avis_client}")


Le client C001 a laissé le plus d’avis : 8


In [16]:
#Nombre d'avis par produit
avis_par_produit = df["produit_id"].value_counts()

print("Nombre total d’avis par produit :")
print(avis_par_produit)


Nombre total d’avis par produit :
P001    18
P004    12
P005     9
P003     6
P002     5
Name: produit_id, dtype: int64
